In [3]:
### optimization of buy and sell period lengths based on returns data

# interactive figures
%matplotlib widget 

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import scipy.optimize as opt

In [4]:
sp_500 = pd.read_csv('../test_data/GSPC.csv')
sp_500

,Date,Open,High,Low,Close,Adj Close,Volume
0,1950-01-03,16.660000,16.660000,16.660000,16.660000,16.660000,1260000
1,1950-01-04,16.850000,16.850000,16.850000,16.850000,16.850000,1890000
2,1950-01-05,16.930000,16.930000,16.930000,16.930000,16.930000,2550000
3,1950-01-06,16.980000,16.980000,16.980000,16.980000,16.980000,2010000
4,1950-01-09,17.080000,17.080000,17.080000,17.080000,17.080000,2520000
...,...,...,...,...,...,...,...
17213,2018-05-31,2720.979980,2722.500000,2700.679932,2705.270020,2705.270020,4235370000
17214,2018-06-01,2718.699951,2736.929932,2718.699951,2734.620117,2734.620117,3684130000
17215,2018-06-04,2741.669922,2749.159912,2740.540039,2746.870117,2746.870117,3376510000
17216,2018-06-05,2748.459961,2752.610107,2739.510010,2748.800049,2748.800049,3517790000


In [5]:
# some misc data
x = sp_500['Open'][-2000:]
dates = sp_500['Date'][-2000:]
# percentage returns
x_pct = x.pct_change().fillna(0)
x_pct

# create new df hold both
df = pd.DataFrame(columns=['Dates','Open','pct_change','pct_change_cumprod']) # ,'log_transform'
df['Dates'] = dates
df['Open'] =  x
df['pct_change'] = x_pct
df['pct_change_cumprod'] = (x_pct + 1).cumprod()
# df['log_transform'] = np.log(df['Open'] )

df.reset_index(inplace=True,drop=True)

# plot
df.plot(subplots=True,sharex=True,figsize=(7,7))
plt.tight_layout()

# view data
df

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

,Dates,Open,pct_change,pct_change_cumprod
0,2010-06-28,1077.500000,0.000000,1.000000
1,2010-06-29,1071.099976,-0.005940,0.994060
2,2010-06-30,1040.560059,-0.028513,0.965717
3,2010-07-01,1031.099976,-0.009091,0.956937
4,2010-07-02,1027.650024,-0.003346,0.953736
...,...,...,...,...
1995,2018-05-31,2720.979980,0.006864,2.525271
1996,2018-06-01,2718.699951,-0.000838,2.523155
1997,2018-06-04,2741.669922,0.008449,2.544473
1998,2018-06-05,2748.459961,0.002477,2.550775


In [6]:
# build algorithm

'''
Start with a lump sum to invest, at each timestep choose: sell, hold or buy (if you had sold already). You never put new money in.

This function is runs given four hyperparameters:

# up or down / buy or sell percentage changes of returns 
- down_threshold < 0 
- up_threshold > 0

# how long should the up and down trends have continued for, must be greater than 2 two days ie start the analysis 2 days into the time series
- down_period_float
- up_period_float

The function services as the lost function for an optimization problem which would tweek the hyperparameters to maximise gain.

'''

def buy_sell(data,down_threshold,up_threshold,down_period_float,up_period_float,current_value):

    # store decisions through time
    decisions_over_time = []

    # turn floats to int
    down_period_int = int(down_period_float)
    up_period_int = int(up_period_float)

    # need enough data for the long period check at the first timestemp we start. therefore we start the longest period check
    first_day = down_period_int

    if first_day < up_period_int:
        first_day = first_day

    # step through time series and perform logic as determined by optimizer
    prev_decision = 0 # start with a hold
    for day in range(first_day,len(data[:]):

        # available time window
        avail_data = data[0:day]

        # Check conditions and make a buy sell or hold choice:
        
        #  you can only make a sell chose if your last decision was buy or hold
        if prev_decision == 1 or prev_decision == 0:

            # check conditions: if mean change is blah
            if sum(avail_data[-down_period_int:])/len(avail_data[-down_period_int:]) >= down_threshold: # over time it has averaged more negative than sell threshold - so sell
                decision = -1 # sell
                else:
            decision = 0 # hold

        # you can only make a buy choose if your last decision was sell
        else:
            if sum(avail_data[-up_period_int:])/len(avail_data[-up_period_int:]) >= up_threshold: # over time it has averaged more postive than buy threshold - so buy
                decision = 1 # buy

            else:
                decision = 0 # hold

        decisions_over_time.append(decision)
        
        

SyntaxError: EOF while scanning triple-quoted string literal (<ipython-input-6-864ee8c12a95>, line 14)

In [ ]:
''